In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
import seaborn as sns

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")

In [3]:
cal=["LotShape","HouseStyle","MasVnrType","ExterQual","Foundation","BsmtQual","BsmtFinType1","HeatingQC","KitchenQual","FireplaceQu","GarageFinish"]

In [4]:
x_train=df_train.loc[:,cal]
x_test=df_test.loc[:,cal]
x_test.isnull().sum()

LotShape          0
HouseStyle        0
MasVnrType       16
ExterQual         0
Foundation        0
BsmtQual         44
BsmtFinType1     42
HeatingQC         0
KitchenQual       1
FireplaceQu     730
GarageFinish     78
dtype: int64

In [5]:
x_train.loc[x_train.isnull()["GarageFinish"],:] = x_train.loc[x_train.isnull()["GarageFinish"],:].fillna(value="None")
x_train.loc[x_train.isnull()["FireplaceQu"],:] = x_train.loc[x_train.isnull()["FireplaceQu"],:].fillna(value="None")
x_train.loc[x_train.isnull()["BsmtQual"],:] = x_train.loc[x_train.isnull()["BsmtQual"],:].fillna(value="None")
x_train.loc[x_train.isnull()["BsmtFinType1"],:] = x_train.loc[x_train.isnull()["BsmtFinType1"],:].fillna(value="None")
x_train.loc[x_train.isnull()["MasVnrType"],:] = x_train.loc[x_train.isnull()["MasVnrType"],:].fillna(value="None")

In [6]:
x_test.loc[x_test.isnull()["KitchenQual"],:] = x_test.loc[x_test.isnull()["KitchenQual"],:].fillna(value="None")
x_test.loc[x_test.isnull()["GarageFinish"],:] = x_test.loc[x_test.isnull()["GarageFinish"],:].fillna(value="None")
x_test.loc[x_test.isnull()["FireplaceQu"],:] = x_test.loc[x_test.isnull()["FireplaceQu"],:].fillna(value="None")
x_test.loc[x_test.isnull()["BsmtQual"],:] = x_test.loc[x_test.isnull()["BsmtQual"],:].fillna(value="None")
x_test.loc[x_test.isnull()["BsmtFinType1"],:] = x_test.loc[x_test.isnull()["BsmtFinType1"],:].fillna(value="None")
x_test.loc[x_test.isnull()["MasVnrType"],:] = x_test.loc[x_test.isnull()["MasVnrType"],:].fillna(value="None")

In [7]:
x_train.shape

(1460, 11)

In [8]:
x_test.shape

(1459, 11)

In [9]:
add_x = pd.concat([x_train,x_test])
one_hot = pd.get_dummies(add_x)
train_onehot=one_hot.iloc[0:1460,:]
test_onehot=one_hot.iloc[1460:2920,:]

---

In [10]:
from sklearn.preprocessing import MinMaxScaler 

In [11]:
x_data =df_train.loc[:,["OverallQual","YearBuilt","YearRemodAdd","MasVnrArea","TotalBsmtSF","1stFlrSF","GrLivArea","FullBath","TotRmsAbvGrd","Fireplaces","GarageCars","GarageArea"]]

In [12]:
x_data.isnull().sum().sort_values(ascending=False)

MasVnrArea      8
GarageArea      0
GarageCars      0
Fireplaces      0
TotRmsAbvGrd    0
FullBath        0
GrLivArea       0
1stFlrSF        0
TotalBsmtSF     0
YearRemodAdd    0
YearBuilt       0
OverallQual     0
dtype: int64

In [13]:
x_data.loc[x_data.isnull()["MasVnrArea"],:] = x_data.loc[x_data.isnull()["MasVnrArea"],:].fillna(value=x_data["MasVnrArea"].mean())

In [15]:
x_data_train = pd.concat([train_onehot,x_data], axis=1)

In [16]:
scaler = MinMaxScaler()
scaler.fit(x_data_train)
x_data =scaler.transform(x_data_train)

In [17]:
y_data = df_train.iloc[:,-1]
y_data = y_data.values
y_data=y_data.reshape(-1,1)

In [18]:
print(x_data.shape)
print(y_data.shape)

(1460, 70)
(1460, 1)


In [20]:
x_data

array([[0.        , 0.        , 0.        , ..., 0.        , 0.5       ,
        0.3864598 ],
       [0.        , 0.        , 0.        , ..., 0.33333333, 0.5       ,
        0.32440056],
       [1.        , 0.        , 0.        , ..., 0.33333333, 0.5       ,
        0.42877292],
       ...,
       [0.        , 0.        , 0.        , ..., 0.66666667, 0.25      ,
        0.17771509],
       [0.        , 0.        , 0.        , ..., 0.        , 0.25      ,
        0.16925247],
       [0.        , 0.        , 0.        , ..., 0.        , 0.25      ,
        0.19464034]])

In [28]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(tf.random_normal([70,1]),name='weight1')
b1 = tf.Variable(tf.random_normal([1]),name='bias1')
hypothesis = tf.matmul(X, W)+b1
#W2 = tf.Variable(tf.random_normal([70,1]),name='weight2')
#b2 = tf.Variable(tf.random_normal([1]),name='bias2')
#hypothesis = tf.matmul(layer1, W2)+b2

cost = -tf.reduce_mean(Y * tf.log(hypothesis)+(1-Y)*tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate= 0.1).minimize(cost)

predicted = tf.cast(hypothesis > 0.5 , dtype =tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype= tf.float32))

In [32]:
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    
    for step in range(200000):
        sess.run(train,feed_dict={X :x_data, Y:y_data})
        if step % 10000 == 0:
            print(step,sess.run(cost, feed_dict={X:x_data, Y:y_data}),sess.run(W))
        
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis: ",h,"\nCorrect:",c,"\nAccuaracy:",a)

0 nan [[-1.58716875e+05]
 [-1.25488770e+03]
 [-1.65146866e+02]
 [ 5.47047510e+03]
 [ 2.02976050e+03]
 [ 5.25473206e+02]
 [-9.37938965e+02]
 [-6.91733246e+01]
 [-2.09722809e+02]
 [-1.53716203e+05]
 [ 9.35913513e+02]
 [-3.22361987e+03]
 [-5.44394455e+01]
 [ 1.52871082e+03]
 [-1.59486609e+05]
 [ 3.34326855e+03]
 [ 1.34360791e+03]
 [ 3.02042904e+01]
 [-1.51645250e+05]
 [-4.39780957e+03]
 [-5.51066772e+02]
 [-3.35596826e+03]
 [-1.51328297e+05]
 [-2.22609196e+01]
 [ 6.09405212e+02]
 [-2.10820141e+01]
 [ 3.64540552e+03]
 [-4.21343899e+00]
 [-1.55651938e+05]
 [ 4.78816254e+02]
 [-3.13204663e+03]
 [ 1.08086815e+01]
 [-3.11547144e+03]
 [ 2.44853955e+03]
 [ 7.03754578e+01]
 [ 4.76083099e+02]
 [-4.48757706e+01]
 [-1.54512469e+05]
 [ 2.25430054e+03]
 [ 9.50896240e+02]
 [-1.53653922e+05]
 [-1.93832035e+01]
 [-4.20127051e+03]
 [ 2.18848120e+03]
 [-9.01237854e+02]
 [ 8.67086853e+02]
 [ 3.49694133e-01]
 [-1.56822812e+05]
 [-5.54859253e+02]
 [ 5.65003540e+02]
 [ 2.96674976e+03]
 [-7.56423145e+03]
 [ 8.0